In [31]:
from theano.sandbox import cuda
cuda.use('gpu0')

In [32]:
import utils; reload(utils)
from utils import *

# Get data and setup

In [110]:
path = get_file('kafka.txt', origin="http://www.gutenberg.org/cache/epub/22367/pg22367.txt")
text = open(path).read()
text = text[1200:-19500]
print('corpus length:', len(text))

('corpus length:', 125146)


In [128]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
chars.insert(0, "\0")
print('total chars:', vocab_size)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

('total chars:', 72)


In [113]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [114]:
text_as_id = [char_indices[c] for c in text]

In [115]:
n_fac = 24
maxlen = 40
batch_size = 64
sentences = []
next_chars = []
for i in range(0, len(text_as_id) - maxlen+1):
    sentences.append(text_as_id[i: i + maxlen])
    next_chars.append(text_as_id[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

('nb sequences:', 125107)


## Building a keras model

In [116]:
vocab_size

72

In [117]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2, consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [118]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [119]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 40, 24)        1728        embedding_input_3[0][0]          
____________________________________________________________________________________________________
lstm_5 (LSTM)                    (None, 40, 512)       1099776     embedding_3[0][0]                
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 40, 512)       0           lstm_5[0][0]                     
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (None, 40, 512)       2099200     dropout_5[0][0]                  
___________________________________________________________________________________________

In [121]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=3, verbose=2)

Epoch 1/3
264s - loss: 1.8423
Epoch 2/3
263s - loss: 1.2238
Epoch 3/3
263s - loss: 1.0474


In [122]:
model.optimizer.lr=0.0001

In [123]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=3, verbose=2)

Epoch 1/3
264s - loss: 0.9503
Epoch 2/3
263s - loss: 0.8893
Epoch 3/3
263s - loss: 0.8467


In [124]:
model.optimizer.lr=0.00001

In [129]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=12, verbose=2)

Epoch 1/12
264s - loss: 0.7909
Epoch 2/12
263s - loss: 0.7696
Epoch 3/12
263s - loss: 0.7531
Epoch 4/12
263s - loss: 0.7375
Epoch 5/12
284s - loss: 0.7244
Epoch 6/12
263s - loss: 0.7128
Epoch 7/12
263s - loss: 0.7015
Epoch 8/12
274s - loss: 0.6919
Epoch 9/12
289s - loss: 0.6818
Epoch 10/12
276s - loss: 0.6733
Epoch 11/12
263s - loss: 0.6656
Epoch 12/12
263s - loss: 0.6576


In [126]:
def print_example(seed_string):
    seed_string = seed_string[:(maxlen)]
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [133]:
print_example("klang nicht sicher, ob nicht mehrere sachen")

klang nicht sicher, ob nicht mehrere sachen
mußte, und die man glauben könne. »Ich meine es genau so, wie ich es
sage,« antwortete Gregor: »Bin schon vielleicht unangenehm? Es kann sofort an, geradeaus zurückzuwandern. Er
staunte über die Riesengröße seiner Stumpfheit an eine
anderen Familie Zeit, sich auf ihre eigenen Leib bis an den Boden
dr�
